<a href="https://colab.research.google.com/github/MarinaSupiot/projet_7/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 6.7 MB/s eta 0:00:00


In [2]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [3]:
! pip install streamlit-shap

In [4]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import plotly.graph_objects as go
import shap
import lightgbm
from streamlit_shap import st_shap





# Создание двух колонок для размещения текста и изображения
col1, col2 = st.columns([2, 10])

# В первой колонке разместите текст
col2.markdown('<p style="font-size:36px; font-weight:bold;">Bienvenue dans l\'application<br>Prêt à dépenser!</p>', unsafe_allow_html=True)

# Во второй колонке разместите изображение
col1.image('/content/drive/MyDrive/Projet 7/Prets_a_depenser.png', width=110)




@st.cache_resource
def load_data_and_model():
    # Считывание данных
    df_test = pd.read_csv("/content/drive/MyDrive/Projet 7/test_preprocess_reduit.csv")

    # Загрузка модели
    with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
        model = pickle.load(file)

    return df_test, model

# Загрузка данных и модели
df_test, model = load_data_and_model()

# Выбор клиента
option = st.sidebar.selectbox(
   "Entrer le numero de client",
   (df_test['SK_ID_CURR'].unique()),
   index=None,
   placeholder="Select le numero de client",
)



# Добавление кнопки "Statut de la demande" в боковую панель
if st.sidebar.button("Statut de la demande"):
    # Здесь вы можете добавить код для обработки нажатия кнопки

    # Открытие модели
    #with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
        #model = pickle.load(file)

    # Фильтрация данных для выбранного клиента
    filtered_df = df_test[df_test['SK_ID_CURR'] == option]

    # Проверка наличия данных для выбранного клиента
    if filtered_df.empty:
        probability_value = None
    else:
        # Преобразование входных данных в 2D массив
        input_data = filtered_df.drop(columns=['SK_ID_CURR']).values

        # Проверка, что данные не пусты и имеют два измерения
        if len(input_data.shape) == 2 and input_data.shape[0] > 0 and input_data.shape[1] > 0:
            # Прогнозирование вероятности
            prediction_probabilities = model.predict_proba(input_data)
            # Применение порога
            prediction = (prediction_probabilities[:, 1] > 0.556).astype(int)

            if prediction == 1:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Désolé, votre demande a été rejetée.</p>', unsafe_allow_html=True)


            else:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Félicitation ! Votre demande est acceptée !</p>', unsafe_allow_html=True)
                st.balloons()

            # Получите значение вероятности
            probability_value = prediction_probabilities[0][1]
        else:
            st.write("Erreur : Les données d'entrée sont vides ou n'ont pas deux dimensions.")

    # Определите цвета для красной и зеленой областей
    red_color = '#FF0000'
    green_color = '#00FF00'

    # Определите границы для красной и зеленой областей
    green_range = (0, 0.556)
    red_range = (0.556, 1)

    # Создайте данные для графика
    fig = go.Figure()

    # Добавьте круглый Gauge, только если значение вероятности определено
    if probability_value is not None:
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=probability_value,
            domain={'x': [0, 1], 'y': [0, 1]},
            gauge={
                'shape': "angular",
                'axis': {'range': [None, 1]},
                'bar': {'color': "darkblue"},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': red_range, 'color': red_color},
                    {'range': green_range, 'color': green_color},
                ],
            }
        ))

        # Обновите макет графика
        fig.update_layout(
            autosize=False,
            margin=dict(l=20, r=20, b=20, t=40),
        )

        # Покажите график
        st.plotly_chart(fig)

option1 = st.sidebar.checkbox("Facteurs d'influence globaux")
option2 = st.sidebar.checkbox("Facteurs d'influence individuels")
option3 = st.sidebar.checkbox("Comparaison avec les autres clients")


@st.cache_data
def compute_shap_values(_explainer_shap, df):
    return _explainer_shap.shap_values(df)

# Проверка состояния чекбокса option1
if option1:
    explainer_shap = shap.TreeExplainer(model)
    df_test_indexed = df_test.set_index('SK_ID_CURR')

    # Проверка, что shap_values не является None
    shap_values = compute_shap_values(explainer_shap, df_test_indexed)
    if shap_values is not None:
        num_columns = st.slider("Sélectionnez le nombre de colonnes", min_value=1, max_value=80, value=10)

        # Вызов функции с кэшированием
        st_shap(shap.summary_plot(shap_values[1], df_test_indexed, show=False, max_display=num_columns))
    else:
        st.write("Erreur : Les valeurs SHAP ne sont pas disponibles.")


if option2:

  explainer_shap = shap.TreeExplainer(model)
  df_test_indexed = df_test.set_index('SK_ID_CURR')
  idx = df_test_indexed.index.get_loc(option)

    # Проверка, что shap_values не является None
  shap_values = compute_shap_values(explainer_shap, df_test_indexed)
  st_shap(shap.plots._waterfall.waterfall_legacy(explainer_shap.expected_value[0], shap_values[0][idx,:], val_sample_set_t.iloc[idx,:], max_display=15))

Writing app.py


In [47]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import plotly.graph_objects as go
import shap
from streamlit_shap import st_shap
from typing import Tuple, Any


# Функция для загрузки данных и модели
@st.cache_resource(show_spinner=False)
def load_data_and_model() -> Tuple[pd.DataFrame, Any]:
    df_test = pd.read_csv("/content/drive/MyDrive/Projet 7/test_preprocess_reduit.csv")
    with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
        model = pickle.load(file)
    return df_test, model


# Функция для вычисления shap_values
@st.cache_data(show_spinner=False)
def compute_shap_values(_explainer_shap, df):
    return _explainer_shap.shap_values(df)


# Загрузка данных и модели
df_test, model = load_data_and_model()

# Создание двух колонок для размещения текста и изображения
col1, col2 = st.columns([2, 10])

# Проверка, находится ли страница в начальном состоянии (при загрузке)
if "is_initial_load" not in st.session_state:
    # В первой колонке разместите текст только при загрузке страницы
    col2.markdown('<p style="font-size:36px; font-weight:bold;">Bienvenue dans l\'application<br>Prêt à dépenser!</p>', unsafe_allow_html=True)
    # Во второй колонке разместите изображение
    col1.image('/content/drive/MyDrive/Projet 7/Prets_a_depenser.png', width=110)
    st.session_state.is_initial_load = True




# Выбор клиента
option = st.sidebar.selectbox(
   "Entrer le numero de client",
   (df_test['SK_ID_CURR'].unique()),
   index=None,
   placeholder="Select le numero de client",
)

# Добавление кнопки "Statut de la demande" в боковую панель
if st.sidebar.button("Statut de la demande"):
    # Фильтрация данных для выбранного клиента
    filtered_df = df_test[df_test['SK_ID_CURR'] == option]

    # Проверка наличия данных для выбранного клиента
    if not filtered_df.empty:
        # Преобразование входных данных в 2D массив
        input_data = filtered_df.drop(columns=['SK_ID_CURR']).values

        # Проверка, что данные не пусты и имеют два измерения
        if len(input_data.shape) == 2 and input_data.shape[0] > 0 and input_data.shape[1] > 0:
            # Открытие модели
            # with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
            #    model = pickle.load(file)

            # Прогнозирование вероятности
            prediction_probabilities = model.predict_proba(input_data)
            # Применение порога
            prediction = (prediction_probabilities[:, 1] > 0.556).astype(int)

            if prediction == 1:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Désolé, votre demande a été rejetée.</p>', unsafe_allow_html=True)
            else:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Félicitation ! Votre demande est acceptée !</p>', unsafe_allow_html=True)
                st.balloons()

            # Получите значение вероятности
            probability_value = prediction_probabilities[0][1]

            # Определите цвета для красной и зеленой областей
            red_color = '#FF0000'
            green_color = '#00FF00'

            # Определите границы для красной и зеленой областей
            green_range = (0, 0.556)
            red_range = (0.556, 1)

            # Создайте данные для графика
            fig = go.Figure()

            # Добавьте круглый Gauge, только если значение вероятности определено
            fig.add_trace(go.Indicator(
                mode="gauge+number",
                value=probability_value,
                domain={'x': [0, 1], 'y': [0, 1]},
                gauge={
                    'shape': "angular",
                    'axis': {'range': [None, 1]},
                    'bar': {'color': "darkblue"},
                    'bgcolor': "white",
                    'borderwidth': 2,
                    'bordercolor': "gray",
                    'steps': [
                        {'range': red_range, 'color': red_color},
                        {'range': green_range, 'color': green_color},
                    ],
                }
            ))

            # Обновите макет графика
            fig.update_layout(
                autosize=False,
                margin=dict(l=20, r=20, b=20, t=40),
            )

            # Покажите график
            st.plotly_chart(fig)
        else:
            st.write("Erreur : Les données d'entrée sont vides ou n'ont pas deux dimensions.")

# Опции
option1 = st.sidebar.checkbox("Facteurs d'influence globaux")
option2 = st.sidebar.checkbox("Facteurs d'influence individuels")
option3 = st.sidebar.checkbox("Comparaison avec les autres clients")

# Создание виджета Spinner
spinner_text = st.empty()
spinner = st.spinner()

# Проверка состояния чекбокса option1
if option1:
    st.title("Facteurs d'influence globaux")
    explainer_shap = shap.TreeExplainer(model)
    df_test_indexed = df_test.set_index('SK_ID_CURR')

    # Проверка, что shap_values не является None
    shap_values = compute_shap_values(explainer_shap, df_test_indexed)
    if shap_values is not None:
        num_columns = st.slider("Sélectionnez le nombre de colonnes", min_value=1, max_value=80, value=10)

        # Запустите Spinner перед выполнением долгой операции
        with spinner:
            # Обновление текста Spinner
            #spinner_text.text("Calcul en cours...")

            # Вызов функции с кэшированием
            st_shap(shap.summary_plot(shap_values[1], df_test_indexed, show=False, max_display=num_columns))
        spinner_text.text("")

    else:
        st.write("Erreur : Les valeurs SHAP ne sont pas disponibles.")
# Проверка опции 2
if option2:
    st.title("Facteurs d'influence individuels")
    explainer_shap = shap.TreeExplainer(model)
    df_test_indexed = df_test.set_index('SK_ID_CURR')
    idx = df_test_indexed.index.get_loc(option)

    # Проверка, что shap_values не является None
    shap_values = compute_shap_values(explainer_shap, df_test_indexed)
    if shap_values is not None:
        # Add a unique key to the slider widget
        num_columns_2 = st.slider("Sélectionnez le nombre de colonnes", min_value=1, max_value=80, value=10, key="slider_option2")
        # Запустите Spinner перед выполнением долгой операции
        with st.spinner("Calcul en cours..."):
            # Обновление текста Spinner
            #spinner_text.text("Calcul en cours...")
            st_shap(shap.plots._waterfall.waterfall_legacy(explainer_shap.expected_value[0], shap_values[0][idx, :], df_test_indexed.iloc[idx, :],  max_display=num_columns_2))
    else:
        st.write("Erreur : Les valeurs SHAP ne sont pas disponibles.")



Overwriting app.py


In [36]:
!wget -q -O - ipv4.icanhazip.com


34.141.160.223


In [49]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session 1648f9c54e1e844


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.160.223:8501

npx: installed 22 in 3.274s
your url is: https://wet-cats-grab.loca.lt
/usr/local/lib/python3.10/dist-packages/shap/explainers/_tree.py:426: UserWarning:

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray

  Stopping...
^C
